## 原方法

In [ ]:
## 顯示 Raw Data 資訊
from scipy import stats
from scipy.stats import norm, skew
# 查看 scaler_train['Loss_mae'] 的 skewness
fig=plt.figure(figsize=(15,5))
# pic1
plt.subplot(1,2,1)
sns.distplot(scored_train['Loss_mae'],fit=norm)
(mu,sigma)=norm.fit(scored_train['Loss_mae'])
plt.legend(['$\mu=$ {:.2f} and $\sigma=$ {:.2f}'.format(mu,sigma)],loc='best')
plt.ylabel('Frequency')
# pic2
plt.subplot(1,2,2)
res=stats.probplot(scored_train['Loss_mae'],plot=plt)
plt.suptitle('Before')
print(f"Skewness of saleprice: {scored_train['Loss_mae'].skew()}")
print(f"Kurtosis of saleprice: {scored_train['Loss_mae'].kurt()}")

In [ ]:
# 3.
# Threshold 決定(評估耗損時間起始點 TSP)
# 將 Threshold 設定為高於數據 99.98%
TSP_threshold = np.quantile(scored_train['Loss_mae'], q=0.9998)
print(f'MSE 0.9998 Threshold: {TSP_threshold}')
print(max(scored_train['Loss_mae']))

# scored['Threshold'] = TSP_threshold

## Box-Cox 轉換

In [ ]:
# 創建 Box-Cox 轉換後 DataFrame
scored_train_BoxCox = pd.DataFrame(index=scored_train.index)

In [ ]:
# 當原始數據分佈是有偏度時，不滿足正態分佈，通過Box-cox轉換修正分佈的偏態
# 使用 Box-cox 來降低 skewness 值，達到接近正態分佈的目的
# power transform
from scipy.stats import boxcox

# 需搜尋最優的參數 lambda
scored_train_BoxCox['Loss_mae'],lambda_=stats.boxcox(scored_train['Loss_mae'])
print(f"λ : {lambda_}")

In [ ]:
## 查看 Box-Cox 轉換後資訊
fig=plt.figure(figsize=(15,5))
#pic1
plt.subplot(1,2,1)
sns.distplot(scored_train_BoxCox['Loss_mae'],fit=norm)
(mu,sigma)=norm.fit(scored_train_BoxCox['Loss_mae'])
plt.legend(['$\mu=$ {:.2f} and $\sigma=$ {:.2f}'.format(mu,sigma)],loc='best')
plt.ylabel('Frequency')
#pic2
plt.subplot(1,2,2)
res=stats.probplot(scored_train_BoxCox['Loss_mae'],plot=plt)
plt.suptitle('After')
print(f"Skewness of saleprice: {scored_train_BoxCox['Loss_mae'].skew()}")
print(f"Kurtosis of saleprice: {scored_train_BoxCox['Loss_mae'].kurt()}")

In [ ]:
# 1.
def threshold_select_BoxCox_test(value, lam):
    # log case
    if lam == 0:
        return np.exp(np.quantile(value, q=0.9998))
    # all other cases
    return (lam*(np.quantile(value, q=0.9998))+1)**(1/lam)
threshold_boxcox_test = threshold_select(scored_train_BoxCox['Loss_mae'], lambda_)
print(threshold_boxcox_test)  

In [ ]:
# 2.
def threshold_select_BoxCox(value, lam):
    # log case
    if lam == 0:
        return np.exp(np.mean(value) + np.std(value)*3)
    # all other cases
    return (lam*(np.mean(value) + np.std(value)*3)+1)**(1/lam)
threshold_boxcox = threshold_select(scored_train_BoxCox['Loss_mae'], lambda_)
print(threshold_boxcox)

## Yeo-Johnson 轉換

In [ ]:
# Yeo-Johnson 轉換
from scipy import stats

# 創建 Yeo-Johnson 轉換後 DataFrame
scored_train_YeoJohnson = pd.DataFrame(index=scored_train.index)

scored_train_YeoJohnson['Loss_mae'],lambda1 = stats.yeojohnson(scored_train['Loss_mae'])
print(f"λ : {lambda1}")

In [ ]:
# Yeo-Johnson 轉換 視覺化
fig=plt.figure(figsize=(15,5))
#pic1
plt.subplot(1,2,1)
sns.distplot(scored_train_YeoJohnson['Loss_mae'],fit=norm)
(mu,sigma)=norm.fit(scored_train_YeoJohnson['Loss_mae'])
plt.legend(['$\mu=$ {:.2f} and $\sigma=$ {:.2f}'.format(mu,sigma)],loc='best')
plt.ylabel('Frequency')
#pic2
plt.subplot(1,2,2)
res=stats.probplot(scored_train_YeoJohnson['Loss_mae'],plot=plt)
plt.suptitle('After')
print(f"Skewness of saleprice: {scored_train_YeoJohnson['Loss_mae'].skew()}")
print(f"Kurtosis of saleprice: {scored_train_YeoJohnson['Loss_mae'].kurt()}")

In [ ]:
def threshold_select_YeoJohnson(value, lam):
    if np.quantile(value, q=0.9998) >= 0 and lam == 0:
        return (exp(np.quantile(value, q=0.9998)) - 1)
    elif np.quantile(value, q=0.9998) >= 0 and lam != 0:
        return ((np.quantile(value, q=0.9998) * lam + 1) ** (1 / lam) - 1)
    elif np.quantile(value, q=0.9998) < 0 and lam != 2:
        return (1 - (-(2 - lam) * np.quantile(value, q=0.9998) + 1) ** (1 / (2 - lam)))
    elif np.quantile(value, q=0.9998) < 0 and lam == 2:
        return (1 - exp(-np.quantile(value, q=0.9998)))

threshold_YeoJohnson = threshold_select_YeoJohnson(scored_train_YeoJohnson['Loss_mae'], lambda1)
print(threshold_YeoJohnson)